<a href="https://colab.research.google.com/github/sverdoot/DL-in-NLP-course/blob/master/workshop%203/task2_char_language_model_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Посимвольная языковая модель. Часть 2.

В этом задании Вам нужно использовать написанную языковую модель для генерации воображаемых какслов.

**В процессе написания Вам нужно решить следующие проблемы**:
    
* как запоминать изменившееся состояние модели и передавать его на следующий временной шаг.
* что будет начальным состоянием модели.
* как понять, что слово закончилось и нужно прекратить генерацию.

**Результаты**:

* генератор слов на основе обученной посимвольной модели
* посимвольные вероятности сгенерированных слов

**Дополнительно**:

* ускорение модели за счёт побатчевой генерации

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
%matplotlib inline
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import math, copy, time
import matplotlib.pyplot as plt
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from IPython.core.debugger import set_trace
from torch import optim
import itertools

USE_CUDA = torch.cuda.is_available()
device=torch.device('cuda:0') # or set to 'cpu'
print("CUDA:", USE_CUDA)
print(device)

seed = 42
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)

import re
from matplotlib import pyplot as plt

CUDA: True
cuda:0


In [0]:
def read_dataset(infile):
    words, tags = [], []
    with open(infile, "r", encoding="utf8") as f:
        for line in f:
            line = line.strip()
            splitted = line.split("\t")
            if len(splitted) != 3:
                continue
            words.append(splitted[0].lower())
            tags.append(splitted[2])
    return words, tags

train_words, train_tags = read_dataset("/content/drive/My Drive/Colab Notebooks/DL in NLP/Work3/russian-train-high")
dev_words, dev_tags = read_dataset("/content/drive/My Drive/Colab Notebooks/DL in NLP/Work3/russian-dev")
test_words, test_tags = read_dataset("/content/drive/My Drive/Colab Notebooks/DL in NLP/Work3/russian-covered-test")

In [0]:
def make_data(words):
    data = [re.findall(r'^(\w+)', re.sub(r'<.*>', "", x))[0] for x in train_words]
    data = list(filter(lambda x: re.match(r'^[а-я]', x), data))
    return data
train_data = make_data(train_words)
eval_data = make_data(dev_words)
test_data = make_data(test_words)

In [0]:
max_len = np.max([len(x) for x in train_data])
max_len

23

In [0]:
#START_TOKEN = '<START>'
EOS_TOKEN = '<EOS>'
UNK_TOKEN = '<UNK>'
PAD_TOKEN = '<PAD>'
AUXILIARY = [PAD_TOKEN, EOS_TOKEN]

In [0]:
class Vocabulary:
    def fit(self, data):
        """Extract unique symbols from the data, make itos (item to string) and stoi (string to index) objects"""
        symbols = set(x for elem in data for x in elem)
        self._symbols = AUXILIARY + sorted(symbols)
        
        # Запомните следующую строчку кода - она нужна примерно всегда
        self._symbol_codes = {s: i for i, s in enumerate(self._symbols)}
        self._codes_symbol = {i: s for i, s in enumerate(self._symbols)}
        print(self._symbols)
        print(self._symbol_codes)
        return self

    def __len__(self):
        return len(self._symbols)

    def transform(self, data):
        """Transform data to indices
        Input:
            - data, list of strings
        Output:
            - list of list of char indices

        >>> self.transform(['word1', 'token2'])
        >>> [[24, 2, 19, 13, 3], [8, 2, 9, 1, 7, 4]]
        """
        
        data_ind = [[self._symbol_codes[x] for x in word] for word in data]
        return data_ind
    
    def tok2idx(self, tok):
        return self._symbol_codes[tok]
    
    def idx2tok(self, idx):
        return self._codes_symbol[idx]

In [0]:
voc = Vocabulary()
voc.fit(train_data)
voc.__len__()

['<PAD>', '<EOS>', 'а', 'б', 'в', 'г', 'д', 'е', 'ж', 'з', 'и', 'й', 'к', 'л', 'м', 'н', 'о', 'п', 'р', 'с', 'т', 'у', 'ф', 'х', 'ц', 'ч', 'ш', 'щ', 'ъ', 'ы', 'ь', 'э', 'ю', 'я', 'ё']
{'<PAD>': 0, '<EOS>': 1, 'а': 2, 'б': 3, 'в': 4, 'г': 5, 'д': 6, 'е': 7, 'ж': 8, 'з': 9, 'и': 10, 'й': 11, 'к': 12, 'л': 13, 'м': 14, 'н': 15, 'о': 16, 'п': 17, 'р': 18, 'с': 19, 'т': 20, 'у': 21, 'ф': 22, 'х': 23, 'ц': 24, 'ч': 25, 'ш': 26, 'щ': 27, 'ъ': 28, 'ы': 29, 'ь': 30, 'э': 31, 'ю': 32, 'я': 33, 'ё': 34}


35

In [0]:
PAD_IDX = voc.tok2idx(PAD_TOKEN)
EOS_IDX = voc.tok2idx(EOS_TOKEN)
#SOS_IDX = voc.tok2idx(START_TOKEN)

In [0]:
class Batcher:
  
  def __init__(self, batch_size, data, data_idx=0):
    self.batch_size = batch_size
    self.data = data
    self.data_idx = data_idx
    
  def __iter__(self):
  
    return self

  def __next__(self, data_idx_=None):
    batch_size = self.batch_size
    data = self.data
    global data_idx
    batch = []
    target = []
    lengths =[]
    whole = []

    n_timestep = np.max([len(data[idx % len(data)]) for idx in range(self.data_idx, self.data_idx + batch_size)])
    
    for i in range(batch_size):
      word = data[self.data_idx]
      x = voc.transform([word])[0]
      y = x[1:] + [EOS_IDX] + [PAD_IDX] * (n_timestep - len(word))
      #y = x[1:] + [PAD_IDX] * (n_timestep - len(word) + 1)
      x.append(EOS_IDX)
      x = x + [PAD_IDX] * (n_timestep - len(word))
      
      whole.append((x,
                    y,
                    len(word)))
      self.data_idx = (self.data_idx + 1) % len(data)
      
    whole = sorted(whole, key=lambda x: x[2], reverse=True)
    batch = [x[0] for x in whole]
    target = [x[1] for x in whole]
    lengths = [x[2] for x in whole]
    return (batch, target, lengths)

In [0]:
class Model(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, num_layers=1, dropout=0.):
        super(Model, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.letter_in_embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers, bidirectional=False, dropout=dropout)

        # The linear layer that maps from hidden state space to tag space
        self.letter_out_embeddings = nn.Linear(hidden_dim, vocab_size)

    def forward(self, word, hidden): #lens):
        batch_size = word.size(0)
        seq_len = word.size(1)
        embeds = self.letter_in_embeddings(word)
        #packed = pack_padded_sequence(embeds, lens, batch_first=True)
        if hidden is not None:
            output, final = self.lstm(embeds.view(seq_len, batch_size, -1), hidden)
        else:
            output, final = self.lstm(embeds.view(seq_len, batch_size, -1))
        #output, _ = pad_packed_sequence(output, batch_first=True)
        outs = self.letter_out_embeddings(output)
        tag_scores = F.log_softmax(outs, dim=-1)
        return tag_scores, final
    
    def init_hidden(self, batch_size=1):
        return torch.zeros(self.num_layers, batch_size, self.hidden_dim, device = device), torch.zeros(self.num_layers, batch_size, self.hidden_dim, device = device)
        #return torch.zeros(self.num_layers, batch_size, self.hidden_dim, device = device), torch.zeros(self.num_layers, batch_size, self.hidden_dim, device = device)

In [0]:
def data_gen(data, batch_size, num_batches, data_idx=0):
    batcher = Batcher(batch_size, data, data_idx)
    build_batch = iter(batcher)
    for i in range(num_batches):
        yield next(build_batch)

In [0]:
model = Model(embedding_dim=voc.__len__(), hidden_dim=512, vocab_size=voc.__len__(), num_layers=4, dropout=0.1).to(device=device)

In [0]:
model.load_state_dict(torch.load("/content/drive/My Drive/Colab Notebooks/DL in NLP/Work3/LSTM"))

In [0]:
model.eval()

Model(
  (letter_in_embeddings): Embedding(35, 35)
  (lstm): LSTM(35, 512, num_layers=4, dropout=0.1)
  (letter_out_embeddings): Linear(in_features=512, out_features=35, bias=True)
)

##Генерация слов

In [0]:
def gen_start(batch_size, vocab_size):
    return torch.randint(2, vocab_size, (batch_size, 1))
  
def gen_words(model, vocab_size, n_iter=1, batch_size=16, start_data=None):
    if start_data is not None:
        batch_size = min(len(start_data), batch_size)
        n_iter = int(math.ceil(len(start_data) / batch_size))
    end = torch.LongTensor([EOS_IDX] * batch_size).view(batch_size, 1).to(device=device)
    outs  = []
    print(n_iter)
    for i in range(n_iter):
        concat  = []
        #start_hidden = model.init_hidden(batch_size)
        hidden = None
        out = None
        if start_data is not None:
            if i == n_iter - 1:
                cur_batch_size = len(start_data) - i * batch_size
                end = torch.LongTensor([EOS_IDX] * cur_batch_size).view(cur_batch_size, 1).to(device=device)
            else:
                cur_batch_size = batch_size
            inp = start_data[i * batch_size : i * batch_size + cur_batch_size]
        else:
            inp = gen_start(batch_size, vocab_size).to(device=device)

        for j in range(inp.size(1)):
            concat.append(inp[:, j].view(cur_batch_size, 1))
            
        prev_out = inp
        out, hidden = model(inp, hidden)
        while (not torch.equal(prev_out, end)):
            #out, hidden = model(inp, hidden)
            #inp = torch.argmax(out[-1], dim=1).view(cur_batch_size,-1)
            prev_out, out, hidden = beam_search(model, hidden, out, cur_batch_size)
            concat.append(prev_out)

        outs.append(torch.stack(concat).cpu().detach().numpy())
    return outs

###Реализуем beam search

In [0]:
def beam_search(model, hidden, scores, batch_size, beam_size=3):
    inp = []
    out = []
    upd_hidden0 = []
    upd_hidden1 = []
    max_sc = []
    for i in range(beam_size):
        inp.append(torch.argmax(scores[-1], dim=1).view(batch_size, 1))
        for j in range(batch_size):
            scores[-1, j, inp[i][j, 0]] = 0
        out.append(None)
        upd_hidden0.append(None)
        upd_hidden1.append(None)
        out[i], h = model(inp[i], hidden)
        upd_hidden0[i], upd_hidden1[i] = h
        max_sc.append(torch.max(out[i][-1], dim=1)[0].view(batch_size,-1))
    argmax = torch.argmax(torch.stack(max_sc), dim=0)
    new_inp = torch.stack([torch.stack(out)[argmax[i], :, i, :] for i in range(batch_size)]).view(1, batch_size, -1).to(device=device)
    prev_out = torch.tensor([torch.stack(inp)[argmax[i], i, :] for i in range(batch_size)]).view(batch_size, 1).to(device=device)
    upd_hidden0 = torch.stack([torch.stack(upd_hidden0)[argmax[i], :, i, :] for i in range(batch_size)]).view(hidden[0].size())
    upd_hidden1 = torch.stack([torch.stack(upd_hidden1)[argmax[i], :, i, :] for i in range(batch_size)]).view(hidden[1].size())

    return prev_out, new_inp, (upd_hidden0, upd_hidden1)

####Генерация по первой букве

In [0]:
start_data = torch.LongTensor([[int(i)] for i in range(2, voc.__len__())]).to(device=device)

In [0]:
outs = gen_words(model, voc.__len__(), batch_size=1, start_data=start_data)

33


In [0]:
start_data_np = start_data.cpu().detach().numpy()
for x in outs: 
    for i in range(x.shape[1]):
        idx_word = [b[i] for b in x]
        let_word = [voc.idx2tok(idx[0]) for idx in idx_word]
        str_word = re.findall(r'^\w+', ''.join(let_word))[0]
        print(str_word[0],' : ', str_word)

а  :  абсорбирование
б  :  безотказный
в  :  высокоэффективный
г  :  грузность
д  :  долговременный
е  :  единичный
ж  :  железобетонный
з  :  заклеймить
и  :  изобразить
й  :  йоркширский
к  :  константа
л  :  лесопилка
м  :  маньчжурский
н  :  неприятель
о  :  оборониться
п  :  прислонять
р  :  распростирать
с  :  страхование
т  :  транспортный
у  :  усиленный
ф  :  фанерный
х  :  христадельфианский
ц  :  церковнославянский
ч  :  черепица
ш  :  шарахать
щ  :  щёлкнуть
ъ  :  ъососина
ы  :  ый
ь  :  ь
э  :  экстремизм
ю  :  ювелирный
я  :  ярмо
ё  :  ёжный


####Генерация по первым двум буквам

In [0]:
start_data = torch.LongTensor([[int(i), int(j)] for i, j in itertools.product(range(2, voc.__len__()), range(2, voc.__len__()))]).to(device=device)
outs = gen_words(model, voc.__len__(), batch_size=1, start_data=start_data)
start_data_np = start_data.cpu().detach().numpy()
first2 = {}
for x in outs: 
    for i in range(x.shape[1]):
        idx_word = [b[i] for b in x]
        let_word = [voc.idx2tok(idx[0]) for idx in idx_word]
        str_word = re.findall(r'^\w+', ''.join(let_word))[0]
        first2[str_word[0]+str_word[1]] = str_word
        print(str_word[0], str_word[1], ' : ', str_word)

1089
а а  :  аангич
а б  :  абсорбирование
а в  :  автоматический
а г  :  агрохимик
а д  :  административный
а е  :  аерон
а ж  :  ажиотаж
а з  :  азер
а и  :  аид
а й  :  айсорский
а к  :  актёрский
а л  :  альтернатива
а м  :  аминокислотный
а н  :  антивоенный
а о  :  аористный
а п  :  апостольский
а р  :  архив
а с  :  астроида
а т  :  атомоход
а у  :  аудитория
а ф  :  афроамериканка
а х  :  ахеменид
а ц  :  ацент
а ч  :  ачеловец
а ш  :  ашиптор
а щ  :  ащиотаж
а ъ  :  аътимат
а ы  :  аысорост
а ь  :  аьсорский
а э  :  аэрология
а ю  :  аюрост
а я  :  аян
а ё  :  аёролитет
б а  :  барахолка
б б  :  ббессильный
б в  :  бвесторопный
б г  :  бгота
б д  :  бдреновый
б е  :  безотказный
б ж  :  бжокнуть
б з  :  бздеть
б и  :  биографический
б й  :  бйрократический
б к  :  бкомный
б л  :  благоустройство
б м  :  бместный
б н  :  бноголюдный
б о  :  болезнь
б п  :  бпомость
б р  :  брезентовый
б с  :  бставать
б т  :  бтрантка
б у  :  бурый
б ф  :  бфренка
б х  :  бхостовый
б ц  :  бцен

####Генерация по произвольному началу cлова

In [0]:
words = ['рапсо', 'перекати', 'возда', 'куро', 'херо', 'сиба', 'откупо', 'гага', 'иллюм', 'акса', 'асаси', 'гено']
for word in words:
    start = torch.LongTensor(voc.transform([word])).to(device=device)
    out = gen_words(model, voc.__len__(), start_data=start)
    for x in out: 
        for i in range(x.shape[1]):
            idx_word = [b[i] for b in x]
            let_word = [voc.idx2tok(idx[0]) for idx in idx_word]
            str_word = re.findall(r'^\w+', ''.join(let_word))[0]
            #first2[str_word[0]+str_word[1]] = str_word
            print(str_word[:len(word)], ' : ', str_word)

1
рапсо  :  рапсоматический
1
перекати  :  перекатить
1
возда  :  воздать
1
куро  :  курома
1
херо  :  херосить
1
сиба  :  сибанитарный
1
откупо  :  откупоривать
1
гага  :  гагарка
1
иллюм  :  иллюморопровод
1
акса  :  аксамит
1
асаси  :  асасиновый
1
гено  :  генотип
